In [288]:
import warnings
import agate
from pathlib import Path
from shutil import copyfile
import os
import csv
from decimal import *

warnings.filterwarnings(action='ignore')

g_table_geos = agate.Table.from_csv('../data/county-geos.csv')

# If we already have a geo column
def join_tables_geo(table_one, table_two, output_fp):
    table_join = table_one.join(table_two, 'geo', 'geo',
                            inner=True)
    
    table_join.exclude(['county2']).to_csv(output_fp)
    
# If we already have a geo column in one spreadsheet
# table_one is the table with the geo column
def join_tables_geo_one(table_one, table_two, classify_column, output_fp):
    table_two_geos = table_two.join(g_table_geos, classify_column, 'county', inner=True)
    table_join = table_one.join(table_two_geos, 'geo', 'geo', inner=True)
    
    table_join.exclude(['county2']).to_csv(output_fp)
    
# Join two tables without the geo column
def join_tables(fp, classify_column, classify_column_two, select_columns, rename_columns):            
    try:
        # Take our global table
        # And our table getting passed through the function
        # give them both geo ids
        # then join to make one big, beautiful table
        table_one = agate.Table.from_csv(fp, encoding='utf-8-sig')
        table_two = agate.Table.from_csv(g_table_merge_fp, encoding='utf-8-sig')
        
        table_one.to_csv('../tmp/table-one.csv')
        table_two.to_csv('../tmp/table-two.csv')
        
        table_geos = g_table_geos.join(table_one, 'county', classify_column,
                                inner=True)
        table_geos_two =  g_table_geos.join(table_two, 'county', classify_column_two,
                                inner=True)
        
        table_geos.to_csv('../tmp/table-geos.csv')
        table_geos_two.to_csv('../tmp/table-geos-two.csv')
        
        table_join = table_geos.join(table_geos_two, 'geo', 'geo')

        # If something didn't join right, print it out
        if table_join:
            table_join_null = table_join.where(lambda row: row[classify_column] == None)

            if table_join_null:
                print("Not everything joined. Table saved to tmp dir.")
                table_join.to_csv('../tmp/table-not-joined.csv')


        print("Saving joined table")
        table_final = table_join.exclude(
            ['geo2', 'county2', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'geo_two', 'geo_two2', 'GISJOIN2']
        )
        
        # If we've include columns to only include
        if select_columns:
            table_final = table_final.select(select_columns)
            
        # If we're renaming columns
        if rename_columns:
            table_final = table_final.rename(column_names = rename_columns)

        table_final.to_csv(g_table_output)
        
    except ValueError:
        return False
    
def dict_check_add(key, cat, add, categories_dict, classification, row):
    try:
        if classification == 'ob-hospitals':
            if cat == 'NAV' or cat == '' or int(cat) == 0:
                cat = 'No OB hospital(s)'
            else:
                cat = 'OB hospital(s)'
        
        dicts = [ categories_dict[cat], categories_dict['Total'] ]
        
        # if key == 'prenatal_eligible':
            # print(categories_dict['Total'])
            
        for c_dict in dicts:
            # Ignore if we don't have full results for county
            if add == '*' or add == '' or add == '-':
                add_up = False
            else:
                add_up = True
                
            if classification == 'ob-hospitals' and key == 'prenatal_eligible':
                add = round( float(row[3]) / (1 - ( (100 -  float(add) ) / 100) ) )
                    
            if add_up:
                if key not in c_dict:
                    c_dict[key] = float(Decimal(add))
                else:
                    c_dict[key] += float(Decimal(add))
    except ValueError:
        return False

In [22]:
year = '2014'

g_table_merge_fp = '../edits/hospitals/01-county-totals-14.csv'
fp = '../edits/mortality-rate/04-rates-population-10-14-urbanization.csv'
classify_column = 'county'
classify_column_two = 'county'
select_columns = ['geo', 'county', 'Obstetrics Beds', 'births_' + year, 'deaths_' + year, 'rate_' + year, 'Urbanization Status']
g_table_output = '../edits/02-mortality-ob-hospitals-14-population.csv'

join_tables(fp, classify_column, classify_column_two, select_columns, False)

Saving joined table


In [68]:
year = '2014'

fp = '../edits/02-mortality-ob-hospitals-14-population.csv'
g_table_merge_fp = '../data/prenatal/raw/' + year + '-county.csv'
classify_column = 'county'
classify_column_two = 'Geographic Area'
select_columns = ['geo', 'county', 'Obstetrics Beds', 'Total Number', 'Total Percent', 'births_' + year, 'deaths_' + year, 'rate_' + year, 'Urbanization Status']
g_table_output = '../edits/02-mortality-prenatal-ob-hospitals-14-population.csv'
rename_columns = {'Geographic Area': 'county', 'Total Number': 'prenatal_' + year, 'Total Percent': 'prenatal_percent_' + year}

join_tables(fp, classify_column, classify_column_two, select_columns, rename_columns)

Saving joined table
